In [1]:
import pandas as pd
import requests
import numpy as np

import urllib.request, json 

In [2]:
import time
import json
import pickle
import Salary
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

In [3]:
not_mapped = pd.read_csv("not_mapped_data_two.csv")
not_mapped = not_mapped.replace(to_replace = r'(Incoming|Summer)', value = "", regex = True)
not_mapped = not_mapped.replace(to_replace = r'(Citi)', value = "Citi Bank", regex = True)

In [4]:
# not_mapped = not_mapped[(not_mapped.jobLocation != "None") & (not_mapped.jobTitle != "None")]
# not_mapped

In [5]:
from operator import add
not_mapped_filter = not_mapped[(not_mapped.jobLocation != "None") & (not_mapped.jobTitle != "None")]
not_mapped_cleaned = not_mapped_filter[not_mapped_filter["jobTitle"].map(not_mapped_filter["jobTitle"].value_counts()) > 3]
not_mapped_cleaner = not_mapped_cleaned[not_mapped_cleaned["jobLocation"].map(not_mapped_cleaned["jobLocation"].value_counts()) > 2]
not_mapped_clean = not_mapped_cleaner[not_mapped_cleaner["company"].map(not_mapped_cleaner["company"].value_counts()>3)].reset_index()
not_mapped_clean = not_mapped_clean.replace(to_replace = ["Greater New York City Area", "New York City Metropolitan Area"], value = "New York City")
not_mapped_clean = not_mapped_clean[(not_mapped_clean.company != "GCA")]
not_mapped_clean = not_mapped_clean[not_mapped_clean.jobLocation!="Hong Kong SAR"]
not_mapped_clean = not_mapped_clean[not_mapped_clean.company!="Hong Kong SAR"]
not_mapped_clean = not_mapped_clean[~not_mapped_clean.jobTitle.str.contains("Intern")]
# not_mapped_clean.replace(to_replace = ["Incoming", "Summer"], value = "")
not_mapped_clean
# not_mapped_clean
# not_mapped_clean["jobLocation"].value_counts()

,index,Unnamed: 0,fullName,company,jobTitle,jobLocation,jobDateRange,school,schoolDegree,schoolDegreeSpec
0,6,10,Brooke Watson,Bain & Company,Associate Consultant,New York City,Oct 2020 – Present,Duke University,Bachelor of Engineering - BE,Biomedical/Medical Engineering
1,11,17,Sahara K.,Deutsche Bank,Investment Banking Analyst,"New York, New York, United States",Aug 2020 – Present,Harvard University,Bachelor of Arts - BA,Economics
2,17,27,Rémi Garcia,Bank of America Merrill Lynch,Investment Banking Analyst,San Francisco Bay Area,Jul 2020 – Present,"University of California, Los Angeles",B.A. Economics - Value Investing Concentratio...,None
3,18,28,Mohammad Bakir,Apple,Data Analyst,San Francisco Bay Area,Nov 2019 – Jul 2020,Georgia Institute of Technology,Master's degree,Computer Science - Machine Learning Specializa...
4,19,29,Anish Shah,Goldman Sachs,Quantitative Strategist,New York City,Feb 2020 – Present,Carnegie Mellon University - Tepper School of ...,Master of Science - MS,Computational Finance
...,...,...,...,...,...,...,...,...,...,...
369,2344,2973,Shawn Kang,Moelis & Company,Investment Banking Analyst,San Francisco Bay Area,Jul 2019 – Present,Queen's University,None,Bachelor of Commerce (Honours)
370,2345,2974,Allan Lao,Facebook,Machine Learning Engineer,"Menlo Park, California, United States",Oct 2020 – Present,University of Illinois at Urbana-Champaign,Doctor of Philosophy - PhD,Physics
371,2352,2982,Kinner Patel,Bank of America,Investment Banking Analyst,"New York, New York",Sep 2020 – Present,Vanderbilt University,Bachelor of Science - BS,Major: Human & Organizational Development - Co...
372,2356,2988,Raizel Y.,Credit Suisse,Investment Banking Analyst,Los Angeles Metropolitan Area,Apr 2020 – Present,The Robert Day School of Economics and Finance...,Master of Arts - MA,Finance


In [6]:
city_list = not_mapped_clean["jobLocation"].tolist()
company = not_mapped_clean["company"].tolist()
role = not_mapped_clean["jobTitle"].tolist()
company_role = []

for i in range(len(company)):
    company_role.append(company[i] + " " + role[i])

In [7]:
username = "shanu.kurapati@gmail.com" # your email here
password = "shanu2001" # your password here

def obj_dict(obj):
    return obj.__dict__
#enddef

def json_export(data, cityName):
    jsonFile = open("Data/" + cityName + ".json", "w")
    jsonFile.write(json.dumps(data, indent=4, separators=(',', ': '), default=obj_dict))
    jsonFile.close()
#enddef

In [8]:
def login(driver, username, password):
    driver.get("http://www.glassdoor.com/profile/login_input.htm")
    try:
        user_field = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "username")))
        pw_field = driver.find_element_by_id("userPassword")
        login_button = driver.find_element_by_name("submit")
        user_field.send_keys(username)
        user_field.send_keys(Keys.TAB)
        time.sleep(1)
        pw_field.send_keys(password)
        time.sleep(1)
        login_button.click()
    except TimeoutException:
        print("TimeoutException! Username/password field or login button not found on glassdoor.com")
#enddef

In [9]:
def init_driver():
    driver = webdriver.Chrome(executable_path = "./chromedriver")
    driver.wait = WebDriverWait(driver, 3)
    return driver
#enddef

In [10]:
def search(driver, city, title):
    driver.get("https://www.glassdoor.com/Salaries/index.htm")
    try:
        title_field = WebDriverWait(driver, 3).until(EC.presence_of_element_located(
            (By.ID, "KeywordSearch")))
#         time.sleep(5)
        city_field = driver.find_element_by_id("LocationSearch")
#         search_btn = driver.find_element_by_id("HeroSearchButton")
        title_field.send_keys(title)
#         title_field.send_keys(Keys.TAB)
        time.sleep(1)
        city_field.send_keys(Keys.BACKSPACE*14)
        city_field.send_keys(city)
        city_field.send_keys(Keys.RETURN)
        time.sleep(1)
#         search_btn.click()
    except TimeoutException:
        print("TimeoutException! city/title field or search button not found on glassdoor.com")
#enddef

In [11]:
import re
def parse_salary(salary):
    if salary=="":
        return "Manual"
        
    ans = []
    range = True
    for i in salary.splitlines():
        if "salaries" in i:
            i = ''
            
        if (re.search(r'([$]\d*K - [$]\d*K)', i) != None):
            if(range):
                ans.append(re.search(r'([$]\d*K - [$]\d*K)', i).group())
                range = False
            continue
        
        i = re.sub(r'\D','',i)
        if(i!='' and len(i)>1):
            ans.append(i)
    
    if ("K" in ans[0]):
        ans[0] = re.sub(r'([K])','000',ans[0])
        bare = re.sub(r'([$])', '', ans[0])
#     ans = (int(ans[0] + "000") + int(ans[1] + "000"))/2
        another = bare.split("-")
        ans = (int(another[0]) + int(another[len(another)-1]))/2
        print(ans)
        return ans
    else:
        print(int(ans[0]))
        return int(ans[0])

In [12]:
def get_data(driver, URL):
    driver.get(URL)
    try:
        salary = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//*[@id='SalariesByCompany']"))).text
    except TimeoutException:
        salary = ""
        
#     salary = driver.find_element_by_xpath("//*[@id='SalariesByCompany']").text
#     print(parse_salary(salary))
    return parse_salary(salary)
#enddef

In [13]:
if __name__ == "__main__":
    driver = init_driver()
    time.sleep(3)
    print("Logging into Glassdoor account ...")
    login(driver, username, password)
    time.sleep(10)
    # search(driver, city, title)
    print("\nStarting data scraping ...")
    data_out = []
    
    for i in range(len(city_list)):
        search(driver, city_list[i], company_role[i])
        appendable = get_data(driver, driver.current_url)
        data_out.append(appendable)

Logging into Glassdoor account ...

Starting data scraping ...
85933
90704
89362
60000.0
119031


KeyboardInterrupt: 

In [ ]:
pwd

In [ ]:
not_mapped_clean["salary"] = data_out
not_mapped_clean["salary"].value_counts()
not_mapped_clean.to_csv(r'/Users/shantakurapati/Desktop/glassdoor2.csv', header=True )

In [ ]:
appendable

In [ ]:
use[15]

In [ ]:
use = appendable.splitlines()
re.search(r'([$]\d*K - [$]\d*K)', use[15]).group()
# print(x)

In [ ]:
ans = []
range = True
for i in appendable.splitlines():
    if "salaries" in i:
        i=''
    
    if (re.search(r'([$]\d*K - [$]\d*K)', i) != None):
        if(range):
            ans.append(re.search(r'([$]\d*K - [$]\d*K)', i).group())
            range = False
        continue
        
    i = re.sub(r'\D','',i)
    if(i!='' and len(i)>1):
        ans.append(i)

# ans
# for i in appendable.splitlines():
    
if ("K" in ans[0]):
    ans[0] = re.sub(r'([K])','000',ans[0])
    bare = re.sub(r'([$])', '', ans[0])
#     ans = (int(ans[0] + "000") + int(ans[1] + "000"))/2
    another = bare.split("-")
    ans = (int(another[0]) + int(another[len(another)-1]))/2
    print(ans)
else:
    print(ans[0])

In [ ]:
import re
def parse_salary(salary):
    ans = []
    range = True
    for i in salary.splitlines():
        if "salaries" in i:
            i = ''
            
        if (re.search(r'([$]\d*K - [$]\d*K)', i) != None):
            if(range):
                ans.append(re.search(r'([$]\d*K - [$]\d*K)', i).group())
                range = False
            continue
        
        i = re.sub(r'\D','',i)
        if(i!='' and len(i)>1):
            ans.append(i)
    
    if ("K" in ans[0]):
        ans[0] = re.sub(r'([K])','000',ans[0])
        bare = re.sub(r'([$])', '', ans[0])
#     ans = (int(ans[0] + "000") + int(ans[1] + "000"))/2
        another = bare.split("-")
        ans = (int(another[0]) + int(another[len(another)-1]))/2
        print(ans)
    else:
        print(ans[0])

In [ ]:
main_url = 'https://www.levels.fyi/comp.html?track=Software%20Engineer'
driver.get(main_url)

In [ ]:
df.to_csv('.csv', encoding="utf-8")